In [104]:
"""This will load a CSV file that details which tiles are marked up I got from Brian"""
import os,sys,csv
import pymongo
mongo_con = pymongo.MongoClient('localhost',27017)
csv_data_file ='data/UDA2_pilot'

In [75]:
## First load the CSV file into a list for convenience
csv_data_list  = []
annotation_list = []
with open(csv_data_file,'r') as fp_in:
    reader = csv.reader(fp_in)
    for row in reader:
        annotation_list.append(row)
print len(annotation_list)       

37988


In [101]:
db_ptr = mongo_con['TileMarkup_DB']

print db_ptr.set_profiling_level(2)
print db_ptr.profiling_info()
#db_ptr['rater_list'].insert_one({'rater_name':'Marghoob'})
print db_ptr.collection_names()

None
[{u'writeConflicts': 0, u'allUsers': [], u'execStats': {}, u'responseLength': 58, u'millis': 0, u'ts': datetime.datetime(2015, 9, 27, 19, 14, 10, 972000), u'client': u'127.0.0.1', u'locks': {u'Global': {u'acquireCount': {u'r': 1L, u'w': 1L}}, u'MMAPV1Journal': {u'acquireCount': {u'w': 1L}}, u'Database': {u'acquireCount': {u'W': 1L}}}, u'numYield': 0, u'command': {u'profile': 2}, u'user': u'', u'keyUpdates': 0, u'ns': u'TileMarkup_DB.$cmd', u'op': u'command'}]
[u'rater_list', u'system.indexes', u'system.profile', u'tileLevelData']


In [102]:
index_line = annotation_list[0]
tileData = []
for a_index, cur_tileAnnotation in enumerate(annotation_list[1:]) :  ## NEED TO SKIP THE INDEX LINE
    tile_markup_dict = {}

    for idx,item in enumerate(index_line):
        cur_item_value = cur_tileAnnotation[idx]
        if cur_item_value == '': cur_item_value = None ## Rather insert a none than a ""
        tile_markup_dict[item] = cur_item_value
    tileData.append(tile_markup_dict)
print  tile_markup_dict

tile_db = db_ptr['tileLevelData']

{'ves_clods': None, 'str_regblackdots': None, 'net_targ': None, 'oth_moth': None, 'ves_atyp': None, 'oth_spoke': None, 'ves_pinkveil': '0.0', 'str_reg_blotch': None, 'study_name': 'UDA2_pilot', 'str_irrstreaks': None, 'ves_hel': None, 'ves_linearbranch': None, 'oth_blgrayglob': None, 'superpixel_id': '234', 'ves_coil': None, 'str_irrblueglob': None, 'str_regblackglob': None, 'str_bluegraydots': None, 'c_red': '0.0', 'str_pseudo': None, 'c_dbrown': '0.0', 'oth_sharpdemarc': None, 'user_id': '54cbe807bae47819d8e4c79a', 'oth_ovoids': None, 'str_regbrglob': None, 'net_atyp': '0.0', 'oth_ulcer': None, 'c_lbrown': '0.0', 'ves_comma': None, 'user_login_name': 'marghooa', 'c_bluegray': None, 'c_white': None, 'ves_serp': None, 'str_regress': None, 'oth_finger': None, 'net_neg': None, 'str_irrbluered': None, 'str_bluewhite': None, 'oth_comedo': None, 'ves_milky': None, 'str_lines': None, 'str_irrblackdots': '0.0', 'str_chrys': '0.0', 'str_hypo_areas': None, 'c_black': None, 'ves_hairpin': None, 

In [103]:
#print mongo_con.server_info()
#http://emptysqua.re/blog/mongo-profiling-hacks/  ingo on profiling
print tile_db.delete_many({})##Keeps indexes but empties colletion

tile_db.insert_many(tileData)
db_ptr.current_op()

{u'inprog': []}

In [160]:
tile_db =  db_ptr['tileLevelData']
#print tileData.delete_many({})##Keeps indexes but empties colletion
#print tile_db.count()

def get_unique_image_list( TileDatabase):
    """This will return a sorted list of images available for the current study"""
    sorted_unique_image_list = sorted(tile_db.distinct('image_name'))
    return sorted_unique_image_list

    ### Get list of unique image names

uil = get_unique_image_list(tile_db) ## Get unique image list
cur_image = uil[7]

    feature_set = [u'ves_clods', u'str_regblackdots', u'net_targ', u'oth_moth', u'str_hypo_areas', u'str_scar', u'str_irrred', u'str_reg_blotch', u'str_strless', u'str_irrstreaks', u'ves_hel', u'ves_linearbranch', u'str_radialstream', u'ves_coil', u'oth_ulcer', u'str_bluegraydots', u'c_red', u'str_pseudo', u'str_irrblackglob', u'c_dbrown', u'c_bluegray', u'net_atyp', u'c_lbrown', u'ves_comma',u'net_neg', u'str_regbrglob', u'c_white', u'ves_serp', u'str_irrblueglob', u'oth_finger', u'str_regress', u'str_irrbluered', u'str_bluewhite', u'oth_comedo', u'ves_milky', u'str_lines', u'ves_dotted', u'str_chrys', u'oth_spoke', u'c_black', u'ves_hairpin', u'str_irrbrglob', u'str_regbrdots', u'str_clod', u'str_peribrown', u'oth_milia', u'net_pseudo', u'ves_atyp', u'oth_blgrayglob', u'str_rhomb', u'ves_linear', u'oth_ovoids', u'ves_polymorph', u'str_regblueglob', u'oth_cereb', u'str_irrbluedots', u'str_regblackglob', u'str_irrbrdots', u'oth_leaf', u'oth_sharpdemarc', u'oth_lacuna', u'c_gray', u'c_blue',  u'str_streaks', u'net_typ', u'str_periblack', u'str_circle', u'ves_pinkveil', u'str_irreg_blotch',u'str_irrblackdots', u'c_bluewhite', u'str_bluegraygran']  ### Should eventually pull this directly from mong

def get_raters_for_image( TileDatabase, image_name):
    """For now i am passing image name and NOT the imageUID should probably switch at some point"""
    raters_for_image = TileDatabase.find({'image_name':image_name}).distinct('user_login_name')
    return raters_for_image

def get_MarkupData_for_ImageFeature( TileDatabase, image_name, feature):
    """returns the tiles marked up for a specific feature"""
    data_for_image = TileDatabase.find({'image_name':image_name})
    ## superpixel ID and user_login_name are the most important
    
    
    ### Need to determine best way to structure this... for now I will do it by tile.. then by rater..
    
    tdff = {}  ##Tile Data foe Feature
     
    for rec in data_for_image:
        for k,v in rec.iteritems():
            if k in  feature_set:  ## I at least know I am looking at a feature... there's other keys in this table
                if v:
                    ### means the value for this feature is not none... i.e. it was marked up
                    cur_sp = rec['superpixel_id']
                    #print cur_sp,rec
                    if cur_sp not in tdff:
                        tdff[cur_sp] = {}
                        tdff[cur_sp][rec['user_login_name']] = v
                    else:
                        ### Need to add data for a second rater...
                        tdff[cur_sp][rec['user_login_name']] = v
                        
            #    print k,v,
            #elif k not in feature_set:
            #    print k,v,
        #print
    return tdff
  
# def get_Features_for_Image( TileDatabase, image_name):
#     """This will get a list of all features that have been marked up for a given image,
#     This really should just query the feature database as its much cleaner"""
#     print TileDatabase.find_one({'image_name':image_name}).keys()
print get_raters_for_image(tile_db,cur_image)
#print get_MarkupData_for_ImageFeature( tile_db, cur_image,'net_atyp')
print cur_image
print get_AllMarkupData_for_Image( tile_db, cur_image)
#meh = tile_db.find({'image_name':cur_image}) 

[u'marghooa', u'carrerac', u'haroldr', u'braunr']
UDA2_pilot_008
{u'str_regblackdots': {u'216': {u'carrerac': u'0.0'}, u'217': {u'carrerac': u'0.0'}, u'214': {u'carrerac': u'0.0'}, u'215': {u'carrerac': u'0.0'}, u'212': {u'carrerac': u'0.0'}, u'213': {u'carrerac': u'0.0'}, u'210': {u'carrerac': u'0.0'}, u'211': {u'carrerac': u'0.0'}, u'165': {u'carrerac': u'0.0'}, u'218': {u'carrerac': u'0.0'}, u'219': {u'carrerac': u'0.0'}, u'133': {u'carrerac': u'0.0'}, u'132': {u'carrerac': u'0.0'}, u'131': {u'carrerac': u'0.0'}, u'130': {u'carrerac': u'0.0'}, u'137': {u'carrerac': u'0.0'}, u'136': {u'carrerac': u'0.0'}, u'135': {u'carrerac': u'0.0'}, u'134': {u'carrerac': u'0.0'}, u'139': {u'carrerac': u'0.0'}, u'138': {u'carrerac': u'0.0'}, u'166': {u'carrerac': u'0.0'}, u'24': {u'carrerac': u'0.0'}, u'25': {u'carrerac': u'0.0'}, u'26': {u'carrerac': u'0.0'}, u'27': {u'carrerac': u'0.0'}, u'20': {u'carrerac': u'0.0'}, u'21': {u'carrerac': u'0.0'}, u'22': {u'carrerac': u'0.0'}, u'23': {u'carrerac':

In [159]:
def get_AllMarkupData_for_Image( TileDatabase, image_name):
    """returns the tiles marked up for a specific feature"""
    data_for_image = TileDatabase.find({'image_name':image_name})
    ## superpixel ID and user_login_name are the most important
    
    
    ### Need to determine best way to structure this... for now I will do it by tile.. then by rater..
    
    tdfaf = {}  ##Tile Data for ALL Features
     
    for rec in data_for_image:
        for k,v in rec.iteritems():
            if k in feature_set:
                if v:
                    ### means the value for this feature is not none... i.e. it was marked up
                    cur_sp = rec['superpixel_id']
                    #print cur_sp,rec
                    if k not in tdfaf:
                        tdfaf[k] = {}  ### Have to create a top level key for the current feature
                    
                    
                    if cur_sp not in tdfaf[k]:  ##Also need to make sure there's a spot for the superpixel
                        tdfaf[k][cur_sp] = {}
                        tdfaf[k][cur_sp][rec['user_login_name']] = v
                    else:
                        ### Need to add data for a second rater...
                        tdfaf[k][cur_sp][rec['user_login_name']] = v
                        
            #    print k,v,
            #elif k not in feature_set:
            #    print k,v,
        #print
    return tdfaf
  